In [25]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('recom.csv', sep=';', encoding='PT154')

In [3]:
df.head()

,order_number,order_date,client_id,item_name,item_quantity,level1,level3
0,198672,2019-05-01,14631,ПД-11-12,"10,90",одежные ткани,подкладочные
1,198672,2019-05-01,14631,ПД-17-3,"10,00",одежные ткани,подкладочные
2,198672,2019-05-01,14631,ПД-17-9,"10,00",одежные ткани,подкладочные
3,198672,2019-05-01,14631,ПД-3-1,"10,00",одежные ткани,подкладочные
4,198672,2019-05-01,14631,ПД-3-16,"10,00",одежные ткани,подкладочные


In [4]:
df['item_quantity'] = df['item_quantity'].replace(
    to_replace=r',', value='.', regex=True).astype('float32')

In [5]:
df.head()

,order_number,order_date,client_id,item_name,item_quantity,level1,level3
0,198672,2019-05-01,14631,ПД-11-12,10.9,одежные ткани,подкладочные
1,198672,2019-05-01,14631,ПД-17-3,10.0,одежные ткани,подкладочные
2,198672,2019-05-01,14631,ПД-17-9,10.0,одежные ткани,подкладочные
3,198672,2019-05-01,14631,ПД-3-1,10.0,одежные ткани,подкладочные
4,198672,2019-05-01,14631,ПД-3-16,10.0,одежные ткани,подкладочные


In [6]:
del df['order_number']
del df['order_date']
del df['level3']

In [7]:
df.head()

,client_id,item_name,item_quantity,level1
0,14631,ПД-11-12,10.9,одежные ткани
1,14631,ПД-17-3,10.0,одежные ткани
2,14631,ПД-17-9,10.0,одежные ткани
3,14631,ПД-3-1,10.0,одежные ткани
4,14631,ПД-3-16,10.0,одежные ткани


In [8]:
df.level1.unique()

array(['одежные ткани', 'ткани для дома', 'швейная фурнитура',
       'готовые изделия', 'технические ткани', 'выкройки online',
       'швейное оборудование', 'мебельные ткани'], dtype=object)

In [8]:
c = df.groupby(['level1', 'client_id', 'item_name'], as_index=False).agg('sum')
c.head()

,level1,client_id,item_name,item_quantity
0,выкройки online,14614,ВКК-1-1,1.0
1,выкройки online,14614,ВКК-1-2,1.0
2,выкройки online,14614,ВКК-1-3,1.0
3,выкройки online,14614,ВКК-1-4,1.0
4,выкройки online,14614,ВКК-1-5,1.0


In [9]:
fabric = c.loc[(c.level1 == 'ткани для дома') | (c.level1 == 'одежные ткани') | (
    c.level1 == 'технические ткани') | (c.level1 == 'мебельные ткани'), ['client_id', 'item_name', 'item_quantity']]
fabric.head()

,client_id,item_name,item_quantity
1937,14795,ГЖ-7-1,29.400000
1938,14795,ГМ-50-1,31.299999
1939,14795,ГМ-55-1,34.000000
1940,14795,РМ-8-1,28.400000
1941,17255,ГМ-51-1,27.000000


In [30]:
fabric['item_name'].unique().shape

(4014,)

In [10]:
def scores(df, id_client):
    b = df.loc[df.client_id == id_client]
    b['score'] = round(b['item_quantity'] / b['item_quantity'].max() * 5)
    return b

In [11]:
res = pd.DataFrame(columns=('client_id', 'item_name', 'item_quantity', 'score'))

In [12]:
clients_list = fabric.client_id.unique()
clients_list.shape

(2539,)

In [13]:
for i in clients_list:
    res = res.append(scores(fabric, i), ignore_index=True)

/home/ssv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
res.shape, fabric.shape

((18107, 4), (18107, 3))

In [15]:
del res['item_quantity']
res.head()

,client_id,item_name,score
0,14795,ГЖ-7-1,1.0
1,14795,ГМ-50-1,1.0
2,14795,ГМ-55-1,1.0
3,14795,РМ-8-1,1.0
4,14795,АКС-1-41,1.0


In [16]:
res['item_name'].unique().shape

(4014,)

In [20]:
df_table = res.pivot_table(index='client_id', columns='item_name', values='score', aggfunc='sum'
#                , fill_value=0
              )

In [21]:
df_table.head()

item_name,АКС-1-11,АКС-1-15,АКС-1-16,АКС-1-19,АКС-1-20,АКС-1-25,АКС-1-26,АКС-1-27,АКС-1-28,АКС-1-30,...,ШП-61-1,ШП-70-1,ШТЕ-15-1,ШТЕ-16-1,ШТЕ-17-1,ШТЕ-18-1,ШТЕ-19-1,ШТЕ-24-1,ШТЕ-28-1,ШТЕ-29-1
client_id,,,,,,,,,,,,,,,,,,,,,
11219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_table.describe()

item_name,АКС-1-11,АКС-1-15,АКС-1-16,АКС-1-19,АКС-1-20,АКС-1-25,АКС-1-26,АКС-1-27,АКС-1-28,АКС-1-30,...,ШП-61-1,ШП-70-1,ШТЕ-15-1,ШТЕ-16-1,ШТЕ-17-1,ШТЕ-18-1,ШТЕ-19-1,ШТЕ-24-1,ШТЕ-28-1,ШТЕ-29-1
count,1.0,6.000000,3.000000,2.000000,3.000000,3.0,1.0,24.000000,8.00000,2.00000,...,1.0,2.000000,14.000000,2.000000,4.00000,3.000000,4.00,5.000000,1.0,4.000000
mean,4.0,3.666667,4.000000,2.500000,3.666667,5.0,3.0,3.541667,3.00000,3.50000,...,2.0,0.500000,2.285714,2.500000,3.00000,3.000000,3.75,1.400000,0.0,3.000000
std,NaN,2.065591,1.732051,0.707107,2.309401,0.0,NaN,1.382473,1.77281,2.12132,...,NaN,0.707107,1.938619,3.535534,2.44949,2.645751,2.50,0.894427,NaN,1.414214
min,4.0,1.000000,2.000000,2.000000,1.000000,5.0,3.0,1.000000,1.00000,2.00000,...,2.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.0,2.000000
25%,4.0,2.000000,3.500000,2.250000,3.000000,5.0,3.0,2.000000,1.75000,2.75000,...,2.0,0.250000,0.250000,1.250000,1.50000,2.000000,3.75,1.000000,0.0,2.000000
50%,4.0,5.000000,5.000000,2.500000,5.000000,5.0,3.0,4.000000,2.50000,3.50000,...,2.0,0.500000,2.500000,2.500000,3.50000,4.000000,5.00,2.000000,0.0,2.500000
75%,4.0,5.000000,5.000000,2.750000,5.000000,5.0,3.0,5.000000,5.00000,4.25000,...,2.0,0.750000,4.000000,3.750000,5.00000,4.500000,5.00,2.000000,0.0,3.500000
max,4.0,5.000000,5.000000,3.000000,5.000000,5.0,3.0,5.000000,5.00000,5.00000,...,2.0,1.000000,5.000000,5.000000,5.00000,5.000000,5.00,2.000000,0.0,5.000000


In [26]:
def pearson(s1, s2):
    s1_correlation = s1-s1.mean()
    s2_correlation = s2-s2.mean()
    return np.sum((s1_correlation*s2_correlation)/np.sqrt(np.sum(s1_correlation**2)*np.sum(s2_correlation**2)))

In [28]:
pearson(df_table['АКС-1-11'], df_table['ШП-61-1'])

0.0

In [29]:
def get_recs(item_name, df, num):
    scores = []
    for items in df.columns:
        if items == item_name:
            continue
        cor = pearson(df[item_name], df[items])
        if np.isnan(cor):
            continue
        else:
            scores.append((items, cor))
            
    scores.sort(key=lambda tup: tup[1], reverse=True)
    return scores[:num]

In [34]:
get_recs('АКС-1-27', df_table,5)

[('ФТН-9-28', 0.3130058),
 ('БХ-92-9', 0.2957627),
 ('ФТН-3-33', 0.2957627),
 ('АКС-1-40', 0.2857501),
 ('АКС-1-15', 0.28119606)]